# Build a Fully Local RAG Application With PostgreSQL, Mistral, and Ollama

## Install & import libraries

The required library `psycopg2` is imported to handle PostgreSQL database operations.


In [ ]:
%pip install psycopg2-binary

In [12]:
import psycopg2

## Dataset Preparation

In this tutorial, we will create dummy data of different locations with their description. 

In [13]:
# the knowledge base
dummy_data = [
    {"title": "Seoul Tower", "content": "Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea."},
    {"title": "Gwanghwamun Gate", "content": "Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea."},
    {"title": "Bukchon Hanok Village", "content": "Bukchon Hanok Village is a Korean traditional village in Seoul with a long history."},
    {"title": "Myeong-dong Shopping Street", "content": "Myeong-dong is one of the primary shopping districts in Seoul, South Korea."},
    {"title": "Dongdaemun Design Plaza", "content": "The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea."}
]

## Database connection component

In [14]:
def connect_db():
    return psycopg2.connect( 
        host = 'localhost',
        database = 'postgres',
        user = 'postgres',
        password = 'password',
        port = '5432' 
    )

## Table creation component
This component creates the documents table if it does not already exist. The table includes columns for `id`, `title`, `content`, and `embedding`. 

In [19]:
with connect_db() as conn:
    with conn.cursor() as cur:
        cur.execute("""
            CREATE TABLE IF NOT EXISTS documents (
                id SERIAL PRIMARY KEY,
                title TEXT,
                content TEXT,
                embedding VECTOR(768)
            );
        """)

## Data insertion component
This component inserts data into the documents table. It first inserts the title and content then updates the embedding field using the [`ollama_embed`](https://github.com/timescale/pgai/blob/main/docs/ollama.md#embed) function from pgai.


In [17]:
with connect_db() as conn:
    with conn.cursor() as cur:
        for doc in dummy_data:
            cur.execute("""
                INSERT INTO documents (title, content, embedding)
                VALUES (
                    %(title)s,
                    %(content)s,
                    ai.ollama_embed(
                            'nomic-embed-text', 
                            concat(%(title)s, ' - ', %(content)s), 
                            host=>'http://ollama:11434'
                    )
                );
            """, doc)

Safechecking the insert information in the database.

In [21]:
with connect_db() as conn:
    with conn.cursor() as cur:    
        cur.execute("""
            SELECT title, content, embedding 
            FROM documents;
        """)

        rows = cur.fetchall()
        for row in rows:
            print(f"Title: {row[0]}\nContent: {row[1]}\nEmbedding: {row[2][0:150]}...\n\n")

Title: Seoul Tower
Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea.
Embedding: [-0.4333513,0.004673723,-3.42062,-0.28108785,0.5006557,0.25992024,0.7378622,-0.00095451507,0.100054085,-0.30185965,-1.1801497,0.625085,-0.018718235,0....


Title: Gwanghwamun Gate
Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea.
Embedding: [-0.24228849,1.9167678,-3.500112,-0.50784695,-0.58251315,0.43372986,0.026797678,0.75811553,-0.0910073,-0.52477586,-0.2247471,1.1845706,-0.089577325,-0...


Title: Bukchon Hanok Village
Content: Bukchon Hanok Village is a Korean traditional village in Seoul with a long history.
Embedding: [-0.722167,0.2099505,-3.6068847,-0.48492905,-0.61543924,-0.32833698,0.32197767,0.4255631,-0.40243146,-0.16431984,-0.9500727,-0.08775711,0.90623933,-0....


Title: Myeong-dong Shopping Street
Content: Myeong-dong is one of the primary shopping districts in Seoul

## Retrieve and generate response component

This component takes a query, embeds it, retrieves the most relevant documents based on cosine similarity, and generates a response using the [`ollama_generate`](https://github.com/timescale/pgai/blob/main/docs/ollama.md#generate) function.

In [22]:
query = "Tell me about landmarks in Seoul"

In [23]:
with connect_db() as conn:
    with conn.cursor() as cur:
        # Embed the query using the ollama_embed function
        cur.execute("""
            SELECT ai.ollama_embed(
                    'nomic-embed-text', 
                    %s, 
                    host=>'http://ollama:11434'
            );
        """, (query,))
        query_embedding = cur.fetchone()[0]

        # Retrieve relevant documents based on cosine distance
        cur.execute("""
            SELECT title, content, 1 - (embedding <=> %s) AS similarity
            FROM documents
            ORDER BY similarity DESC
            LIMIT 3;
        """, (query_embedding,))

        rows = cur.fetchall()
            
        # Prepare the context for generating the response
        context = "\n\n".join([f"Landmark: {row[0]}\nDescription: {row[1]}" for row in rows])

        # Generate the response using the ollama_generate function
        cur.execute("""
            SELECT ai.ollama_generate(
                    'mistral', %s, 
                    host=>'http://ollama:11434'
            );
        """, (f"Query: {query}\nContext: {context}",))
        
        model_response = cur.fetchone()[0]
        print(model_response['response'])



1. Dongdaemun Design Plaza (DDP): The Dongdaemun Design Plaza is a significant architectural landmark located in Seoul, South Korea. It serves as a hub for culture, arts, and shopping, with a distinctive design by renowned British architect, Norman Foster. This urban development project hosts various events throughout the year, making it a must-visit destination for tourists.

2. Seoul Tower (Namsan Tower): The Seoul Tower is another iconic landmark in Seoul, situated atop Namsan Mountain in central Seoul. As both a communication and observation tower, visitors can enjoy panoramic views of the city from its observation deck. The tower is easily accessible through cable cars or hiking trails.

3. Gwanghwamun Gate: Gwanghwamun Gate is the primary and largest gate of Gyeongbokgung Palace in Jongno-gu, Seoul, South Korea. Built during the Joseon Dynasty (1392 - 1910), it serves as a symbolic entrance to one of the Five Grand Palaces of Korea. Visitors can learn about Korea's rich history a